# Student Performance Classification

## Import

In [1]:
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner, Evaluator, Pusher
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os

## Set Variable

In [2]:
PIPELINE_NAME = "axelseancp-pipeline"

#Directory untuk menyimpan artifact yang akan dihasilkan
PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)

In [3]:
DATA_ROOT = "data"

In [4]:
interactive_context = InteractiveContext(pipeline_root=PIPELINE_ROOT)

## Tahapan Data Ingestion

In [6]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name="train", hash_buckets=8),
        example_gen_pb2.SplitConfig.Split(name="val", hash_buckets=2)
    ])
)

example_gen = CsvExampleGen(input_base=DATA_ROOT, output_config=output)

In [8]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 2
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Tahapan Data Validation

### Membuat summary statistics

In [9]:
statistics_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)
interactive_context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 3
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [11]:
interactive_context.show(statistics_gen.outputs["statistics"])

### Membuat data schema

In [12]:
schema_gen = SchemaGen(statistics=statistics_gen.outputs["statistics"])

interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 4
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [13]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'Absences',INT,required,,-
'Age',INT,required,,-
'Ethnicity',INT,required,,-
'Extracurricular',INT,required,,-
'GPA',FLOAT,required,,-
'Gender',INT,required,,-
'GradeClass',FLOAT,required,,-
'Music',INT,required,,-
'ParentalEducation',INT,required,,-


### Mencari anomali pada dataset

In [14]:
example_validator = ExampleValidator(
    statistics=statistics_gen.outputs["statistics"],
    schema=schema_gen.outputs["schema"]
)

interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 5
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

In [15]:
interactive_context.show(example_validator.outputs["anomalies"])

## Tahapan Data Preprocessing

In [ ]:
TRANSFORM_MODULE_FILE = "student_transform.py"

In [ ]:
#%%writefile {TRANSFORM_MODULE_FILE}

from sklearn.preprocessing import StandardScaler
# TODO1: buat normalisasi dengan standardscaler
# TODO2: kalau ada hilangin missing value
# TODO3: drop column studentid karena ga butuh dan pisahin labelnya (gradeclass) dari yang lain